# Citibike Data Visualization (03-2020 to 04-2020)

After extracting and importing the data , the next step is to make sure that the visualization of the data is as accurate as possible and for this data cleaning is very imoprtant. It can be done by identifying erros or corruptions, correcting or deleting them, or manually processing data as needed to prevent the same errors from occuring. 

The first step, here, is to import all the necessary modules and then with the help of the modules import file that has the data. Next is to find the data type which helps not just in the accuracy of the visualization but also to makes the cleaning proccess as simple as possible.

In [1]:
#import necessary modules
import pandas as pd
import numpy as np
import datetime
import math

In [2]:
#read the csv file
df = pd.read_csv("Resources/202003_and_04_tripdata.csv")
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1589,2020-03-01 00:00:03.6400,2020-03-01 00:26:32.9860,224,Spruce St & Nassau St,40.71146364,-74.00552427,3574,Prospect Pl & Underhill Ave,40.6769694,-73.96579,16214,Subscriber,1980,1
1,389,2020-03-01 00:00:16.7560,2020-03-01 00:06:46.0620,293,Lafayette St & E 8 St,40.73020660529954,-73.99102628231049,223,W 13 St & 7 Ave,40.73781509,-73.99994661,29994,Subscriber,1991,2
2,614,2020-03-01 00:00:20.0580,2020-03-01 00:10:34.2200,379,W 31 St & 7 Ave,40.749156,-73.9916,515,W 43 St & 10 Ave,40.76009437,-73.99461843,39853,Subscriber,1991,1
3,597,2020-03-01 00:00:24.3510,2020-03-01 00:10:22.3390,3739,Perry St & Greenwich Ave,40.735918,-74.000939,325,E 19 St & 3 Ave,40.73624527,-73.98473765,42608,Subscriber,1989,1
4,1920,2020-03-01 00:00:26.1120,2020-03-01 00:32:26.2680,236,St Marks Pl & 2 Ave,40.7284186,-73.98713956,3124,46 Ave & 5 St,40.74731,-73.95451,36288,Subscriber,1993,1


In [3]:
#check the number of rows and dtypes of the df
print(df.shape)
print(df.dtypes)

(1751569, 15)
tripduration               object
starttime                  object
stoptime                   object
start station id           object
start station name         object
start station latitude     object
start station longitude    object
end station id             object
end station name           object
end station latitude       object
end station longitude      object
bikeid                     object
usertype                   object
birth year                 object
gender                     object
dtype: object


# Cleaning

According to Citi Bike ridership the data includes:

-Trip Duration (seconds)

-Start Time and Date

-Stop Time and Date

-Start Station Name

-End Station Name

-Station ID

-Station Lat/Long

-Bike ID

-User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)

-Gender (Zero=unknown; 1=male; 2=female)

-Year of Birth

Here, the values of each column were object type. This may not be a problem if all the values had letters or if adding and substracting were not required but since numeric values are of great importance for the visualization, I converted the data types from object to appropriate types.

The time for the duration of trip is recorded at seconds so I coverted it from objects to integers. 

In [4]:
#convert dtype of column tripduration and replace anything that is not a number with 0
df["tripduration"] =pd.to_numeric(df["tripduration"], errors = 'coerce').fillna(0).astype(int)

After converting the values of the column tripduration, I dropped the rows that took less than a minute. Keeping the traffic of New York city in mind it is safe to assume that the riders took at least 2 minutes to get from one station to another. 

Likewise, I dropped the rows that had tripduration of more than 24 hours. It is likely that the bike went for repair and maintainance or it was stolen.

In [5]:
#drop rows that were less than 90 or more than 24 hours
df= df[~((df["tripduration"]<90) |(df["tripduration"]>86400))]

I also dropped the rows that started and ended in the same station with trip duration of less than 5 min. This was done on the assumption that the bike or the rider had mechanical issues and also for my analysis I am only considering the rides that covered some distance from one station to another and not just a ride around the block and back. 

In [6]:
#drop rows that started and ended in the same station less than a 5 min
df= df[~((df["tripduration"]<=300) & (df["start station id"] == df["end station id"]))]

The starttime column was converted to datetime following the format year-month-day hour-min-sec. 

In [7]:
#convert dtype of column starttime, set errors='coerce' for invalid values
df['starttime'] = pd.to_datetime(df['starttime'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

Similarly, stoptime column was also converted to datetime following the format year-month-day hour-min-sec.

In [8]:
#convert dtype of column stoptime, set errors='coerce' for invalid values
df['stoptime'] = pd.to_datetime(df['stoptime'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

In the process of converting the starttime and stoptime, I kept getting type error saying that the format does not match. On further inquiry I came to know that some of the rows had name of the columns as their values instead of the actual data. By setting errors to coerce I  replaced the invalid parsing with NaN and later dropped the rows with NaN value. 

Even though the station ids were not of priority, I decided to convert them from object type to integers so that I could remove any missing data. One other way of removing missing data is replacing the invalid parsing (if any) with 0 and then dropping the columns with ) values.

In [9]:
#convert dtype for column start station id where valid
df["start station id"] =pd.to_numeric(df["start station id"], errors = 'coerce').fillna(0).astype(int)
#drop station id 0
df=df[~(df["start station id"] ==0)]

In [10]:
#convert dtype for column end station id where valid
df["end station id"] =pd.to_numeric(df["end station id"], errors = 'coerce').fillna(0).astype(int)
#drop station id 0
df=df[~(df["end station id"] ==0)]

The location of the stations were very important for my analysis so I converted them from objects to appropritate numeric type by using to_numeric so as not to lose precision. 

In [11]:
#convert dtype for all of the given lat and long
df[["start station latitude", 
    "start station longitude", 
    "end station latitude", 
    "end station longitude"]] = df[["start station latitude", 
                                    "start station longitude",
                                    "end station latitude",
                                    "end station longitude"]].apply(pd.to_numeric)

Using the same conversion method, the vlaues on the column birth year were changed to integers from object and any invalid parsing replaced with 0. 

In [12]:
#convert birth year from object to int
df["birth year"] = pd.to_numeric(df["birth year"], errors = 'coerce').fillna(0).astype(int)

Dropped the rows with  birth year 0 and also the ones the with year before 1930 ( riders who are said to be 90 plus). 

In [13]:
#drop birth year 0 (if any) or year 1930 and less
df= df[~((df["birth year"] == 0) | (df["birth year"] <= 1930))]

Changed the bike id to integers.

In [14]:
#convert bikeid from object to int
df["bikeid"] = pd.to_numeric(df["bikeid"]).astype(int)

Replaced the gender category from numbers to corresponding sex.

In [15]:
#replace numbers with genders under gender column
df['gender'] = df["gender"].map({"0":"Unknown", "1":"Male", "2":"Female"})
#df.head()

Checked to make sure the data type were converted correctly.

In [16]:
df.dtypes

tripduration                        int32
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int32
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int32
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int32
usertype                           object
birth year                          int32
gender                             object
dtype: object

Lastly, I dropped all the rows that had any NaN values. At the end the rows omitted were around 23000 or 1.35% of the original data which is reasonable  enough to get some results for the analysis  and for reliable visualization.

In [17]:
#lastly drop any nan values
df.dropna()
df.reset_index(drop=True)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1589,2020-03-01 00:00:03.640,2020-03-01 00:26:32.986,224,Spruce St & Nassau St,40.711464,-74.005524,3574,Prospect Pl & Underhill Ave,40.676969,-73.965790,16214,Subscriber,1980,Male
1,389,2020-03-01 00:00:16.756,2020-03-01 00:06:46.062,293,Lafayette St & E 8 St,40.730207,-73.991026,223,W 13 St & 7 Ave,40.737815,-73.999947,29994,Subscriber,1991,Female
2,614,2020-03-01 00:00:20.058,2020-03-01 00:10:34.220,379,W 31 St & 7 Ave,40.749156,-73.991600,515,W 43 St & 10 Ave,40.760094,-73.994618,39853,Subscriber,1991,Male
3,597,2020-03-01 00:00:24.351,2020-03-01 00:10:22.339,3739,Perry St & Greenwich Ave,40.735918,-74.000939,325,E 19 St & 3 Ave,40.736245,-73.984738,42608,Subscriber,1989,Male
4,1920,2020-03-01 00:00:26.112,2020-03-01 00:32:26.268,236,St Marks Pl & 2 Ave,40.728419,-73.987140,3124,46 Ave & 5 St,40.747310,-73.954510,36288,Subscriber,1993,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727958,1680,2020-04-30 23:59:00.778,2020-05-01 00:27:01.760,447,8 Ave & W 52 St,40.763707,-73.985162,2021,W 45 St & 8 Ave,40.759291,-73.988597,43888,Subscriber,1989,Male
1727959,1334,2020-04-30 23:59:23.646,2020-05-01 00:21:38.219,3292,5 Ave & E 93 St,40.785785,-73.957481,509,9 Ave & W 22 St,40.745497,-74.001971,36853,Subscriber,1973,Male
1727960,1334,2020-04-30 23:59:33.676,2020-05-01 00:21:48.215,3292,5 Ave & E 93 St,40.785785,-73.957481,509,9 Ave & W 22 St,40.745497,-74.001971,37808,Subscriber,1981,Male
1727961,234,2020-04-30 23:59:56.972,2020-05-01 00:03:51.960,469,Broadway & W 53 St,40.763441,-73.982681,495,W 47 St & 10 Ave,40.762699,-73.993012,20635,Subscriber,1996,Female


# Sorting

Since age was one of the factor that I wanted to work on, I created a new column with ages of the riders as the values. By substracting the birth year column from the period of analysis I was able to get the ages of the rider. Then I removed the birth column since it was not necessary.

In [18]:
#calculate age 
df["age"] = 2020 - df["birth year"]

In [19]:
#drop birth year column
del df["birth year"]

Using the Haversine formula I defined a function to get the distance between the start and end station in miles. The formula and how to use it are available with a quick google search. The only thing to bear in mind is the unit of measurement for the distance. Here, I am using Miles, so the radius of the Earth is set at 3959 (approx.). 

In [20]:
# using haversine to get the distance(mil) between two stations 
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 3959 # radius of Earth in miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
 
    return d

I then used the distance function on each of the locations and created a new column with distance between them as the values.

In [21]:
#create a new column with distance
df["distance(mi.)"] = df.apply(lambda x: distance((x["start station latitude"], x["start station longitude"]), (x["end station latitude"], x["end station longitude"])), axis=1)

Usinf the new distance column and trip duration I then calculated the speed of each rider in miles per hour.

In [22]:
#calcuate the average speed
df["speed(miles per hr)"] = df["distance(mi.)"]/(df["tripduration"]/3600)

In [23]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,gender,age,distance(mi.),speed(miles per hr)
0,1589,2020-03-01 00:00:03.640,2020-03-01 00:26:32.986,224,Spruce St & Nassau St,40.711464,-74.005524,3574,Prospect Pl & Underhill Ave,40.676969,-73.965790,16214,Subscriber,Male,40,3.164534,7.169491
1,389,2020-03-01 00:00:16.756,2020-03-01 00:06:46.062,293,Lafayette St & E 8 St,40.730207,-73.991026,223,W 13 St & 7 Ave,40.737815,-73.999947,29994,Subscriber,Female,29,0.703228,6.508026
2,614,2020-03-01 00:00:20.058,2020-03-01 00:10:34.220,379,W 31 St & 7 Ave,40.749156,-73.991600,515,W 43 St & 10 Ave,40.760094,-73.994618,39853,Subscriber,Male,29,0.772151,4.527271
3,597,2020-03-01 00:00:24.351,2020-03-01 00:10:22.339,3739,Perry St & Greenwich Ave,40.735918,-74.000939,325,E 19 St & 3 Ave,40.736245,-73.984738,42608,Subscriber,Male,31,0.848553,5.116904
4,1920,2020-03-01 00:00:26.112,2020-03-01 00:32:26.268,236,St Marks Pl & 2 Ave,40.728419,-73.987140,3124,46 Ave & 5 St,40.747310,-73.954510,36288,Subscriber,Male,27,2.149965,4.031184


With the help of summary statistics, I was able to see if there were any corrupt data or any outliers that might be misleading. 

In [24]:
round(df.describe(),2)

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,age,distance(mi.),speed(miles per hr)
count,1727963.00,1727963.00,1727963.00,1727963.00,1727963.00,1727963.00,1727963.00,1727963.00,1727963.00,1727963.00,1727963.00
mean,1116.43,1889.50,40.74,-73.98,1882.56,40.74,-73.98,33756.79,40.03,1.25,5.27
std,1870.09,1513.88,0.03,0.02,1514.29,0.03,0.02,8276.76,12.22,1.05,2.50
min,90.00,72.00,40.66,-74.02,72.00,40.66,-74.05,14530.00,16.00,0.00,0.00
25%,417.00,408.00,40.72,-73.99,405.00,40.72,-73.99,29143.00,30.00,0.51,3.93
50%,784.00,2003.00,40.74,-73.98,2002.00,40.74,-73.98,36298.00,37.00,0.95,5.56
75%,1437.00,3374.00,40.76,-73.96,3374.00,40.76,-73.96,40423.00,51.00,1.71,6.92
max,86384.00,3992.00,40.84,-73.90,3992.00,40.84,-73.90,43912.00,89.00,11.57,24.33


From the above summary, there were still some discrepencies. The min distance(mi.) is 0 which is possible if the rider started and ended in the same station. In the earlier cleaning proccess I removed only the ones that had short durations.It is possible that the rider went to a different destination and ended back at the starting station. For this reason I did not make any further changes to it. 

Next is the speed. According to the citi bike website the max speed that the citi bike can reach is 18 miles per hour but can reach up to 20 miles per hour and the speed limit for bikes in New York city is  also 20 miles per hour. So for this reason I dropped all the rows that had speed of more than 20 miles per hour.

In [25]:
#drop the rows with speed more than 20 miles per hour
df = df[~(df["speed(miles per hr)"] >20)]

In [26]:
#check the number of rows reduced
df.shape

(1727943, 17)

In [27]:
#save the cleaned data
df.to_csv("Resources/202003_and_04_tripdata_clean.csv")